# 1 Libraries

In [1]:
from influxdb import DataFrameClient
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates 
import requests 
from datetime import datetime, timedelta
import time
import ast

# 2 Order Book merge functions

# 2.1 Binance

In [27]:
# EVENT_TIME (E)
def create_Binance(part, part_dif, last_row):
    # Pass numbers to float format
    part_dif.loc[:,"Seg"] = part_dif.loc[:,"Seg"].apply(lambda x: ast.literal_eval(x))
    part.loc[:, "Asks"] = part.loc[:,"Asks"].apply(lambda x: dict(list([[i[0],float(i[1])] for i in ast.literal_eval(x)])))
    part.loc[:,"Bids"] = part.loc[:,"Bids"].apply(lambda x: dict(list([[i[0],float(i[1])] for i in ast.literal_eval(x)])))
    part_dif.loc[:, "Asks"] = part_dif.loc[:,"Asks"].apply(lambda x: dict(list([[i[0],float(i[1])] for i in ast.literal_eval(x)])))
    part_dif.loc[:,"Bids"] = part_dif.loc[:,"Bids"].apply(lambda x: dict(list([[i[0],float(i[1])] for i in ast.literal_eval(x)])))

    part["Type"] = "Snapshot"
    part_dif["Type"] = "Dif"
    if type(last_row) == type(None):
        # Gather first snapshot valid
        first = 0
        for i in range(len(part)):
            for j in range(len(part_dif)):
                if int(part_dif.iloc[j,4][1]) <= int(part.iloc[i,3]) and int(part_dif.iloc[j,4][0]) > int(part.iloc[i,3]):
                    first = [i,j]
                    break
                elif int(part_dif.iloc[j,4][1]) > int(part.iloc[i,3]):
                    break
            if first != 0:
                break

    else:
        first = [0,0] 

    # Add rest of snapshots to the dataset
    part_dif["U_Asks"] = ""
    part_dif["U_Bids"] = ""

    firsts = [element[1] for element in part_dif.iloc[1::,4]]
    seconds = [element[0] for element in part_dif.iloc[1::,4]]

    pos = []

    df = part
    for element in df.iloc[:,3]:
        pos.append(next(x[0] for x in enumerate(firsts) if x[1] >  int(element)))
        i = 0
        for element in pos:
            if seconds[pos[i]] >= int(df.iloc[i,3]):
                part_dif.iat[element,6] = df.iloc[i,0]
                part_dif.iat[element,7] = df.iloc[i,1]
            i += 1
    # Rebuild order book for each timestemp we have

    # Start from first snapshot
    part_join = part_dif.iloc[first[1]::,[0,1,4,5,6,7]]
    if type(last_row) != type(None):
        part_join = pd.concat([pd.DataFrame(last_row).T, part_join])

    # Rebuild with differences, keeping snapshots when possible
    for i in range(len(part_join)):
        for j in range(2):
            if part_join.iloc[i,4+j] == "":
                part_join.iat[i,4+j] = part_join.iloc[i-1,4+j].copy()
                if i > 1:
                    if part_join.iloc[i,2][1] != part_join.iloc[i-1,2][0] + 1:
                        print("\n BINANCE ERROR")
                        print("Missing records between " + str(part_join.iloc[i-1,2][0]) + " and " + str(part_join.iloc[i,2][1]) + "\n")
            for key in part_join.iloc[i,j].keys():
                if  key in part_join.iloc[i,4+j]:
                    if part_join.iloc[i,j][key] == 0.0:
                        del part_join.iloc[i,4+j][key]
                    else:
                        part_join.iat[i,4+j][key] = part_join.iloc[i,j][key]
                elif part_join.iloc[i,j][key] != 0.0:
                    part_join.iat[i,4+j][key] = part_join.iloc[i,j][key]
    lrow = part_join.iloc[-1,:]     

    return part_join.iloc[1::,:], lrow

# 2.2 Bitfinex

In [28]:
def create_Bitfinex(part, part_dif, last_row):
    
    part.loc[:,"Asks"] = part.loc[:,"Asks"].apply(lambda x: dict([[i[0],i[2]] for i in ast.literal_eval(x)]) if x != "-" else x)
    part.loc[:,"Bids"] = part.loc[:,"Bids"].apply(lambda x: dict([[i[0],i[2]] for i in ast.literal_eval(x)]) if x != "-" else x)  
    part_dif.loc[:,"Bids"] = part_dif.loc[:,"Bids"].apply(lambda x: float(x)) 
    part_dif.loc[:,"Asks"] = part_dif.loc[:,"Asks"].apply(lambda x: float(x)) 
    part_dif.loc[:,"Seg"] = part_dif.loc[:,"Bids"].apply(lambda x: int(x)) 

    join_part = pd.concat([part[["Asks","Bids", "Host", "Pair"]], part_dif[["Asks","Bids", "Host", "Pair","Seg"]]]).sort_index()
    if type(last_row) != type(None):
        join_part = pd.concat([pd.DataFrame(last_row).T, join_part])
    join_part["U_Asks"] = ""
    join_part["U_Bids"] = ""
    join_part.iat[0,5] = part.iloc[0,0].copy()
    join_part.iat[0,6] = part.iloc[0,1].copy()
    
    current = join_part.iloc[0,3]
    for i in range(1,len(join_part)):
        join_part.iat[i,5] = join_part.iloc[i-1,5].copy()
        join_part.iat[i,6] = join_part.iloc[i-1,6].copy()
        if type(join_part.iloc[i,1]) != type({}) and join_part.iloc[i-1,3] == current:
            if join_part.iloc[i,0] < 0:
                if  join_part.iloc[i,1] in join_part.iloc[i,5]:
                    if join_part.iloc[i,4] == 0 and join_part.iloc[i,0] == -1:
                        del join_part.iloc[i,5][join_part.iloc[i,1]]
                    else:
                        join_part.iat[i,5][join_part.iloc[i,1]] = float(join_part.iloc[i,0])
                else:
                    join_part.iat[i,5][join_part.iloc[i,1]] = join_part.iloc[i,0]
            if join_part.iloc[i,0] > 0:
                if  join_part.iloc[i,1] in join_part.iloc[i,6] and join_part.iloc[i,0] == 1:
                    if join_part.iloc[i,4] == 0:
                        del join_part.iloc[i,6][join_part.iloc[i,1]]
                    else:
                        join_part.iat[i,6][join_part.iloc[i,1]] = join_part.iloc[i,0]
                else:
                    join_part.iat[i,6][join_part.iloc[i,1]] = join_part.iloc[i,0]
        elif type(join_part.iloc[i,1]) == type({}):
            join_part.iat[i,5] = join_part.iloc[i,0].copy()
            join_part.iat[i,6] = join_part.iloc[i,1].copy()
            current = join_part.iloc[0,3]
        else:
            join_part.iat[i,5] = join_part.iloc[i-1,5].copy()
            join_part.iat[i,6] = join_part.iloc[i-1,6].copy()
    
    join_part["U_Asks"] = join_part["U_Asks"].apply(lambda x: dict([[str(key), np.abs(x[key])] for key in x.keys()]))
    lrow = join_part.iloc[-1,:]
    return join_part.iloc[1::,:], lrow

# 2.3 Bithumb

In [29]:
# Timestamp in msg
def create_Bithumb(part, part_dif, last_row):
    # Evaluate strings
    part_dif.loc[:,"Asks"] = part_dif.loc[:,"Asks"].apply(lambda x:[[element[0],float(element[1])] for element in ast.literal_eval(x)] if x != "-" else x)
    part_dif.loc[:,"Bids"] = part_dif.loc[:,"Bids"].apply(lambda x: [[element[0],float(element[1])] for element in ast.literal_eval(x)] if x != "-" else x)
    part.loc[:,"Asks"] =  part.loc[:,"Asks"].apply(lambda x: dict([[element[0],float(element[1])] for element in ast.literal_eval(x)]) if x != "-" else x)
    part.loc[:,"Bids"] =  part.loc[:,"Bids"].apply(lambda x: dict([[element[0],float(element[1])] for element in ast.literal_eval(x)]) if x != "-" else x)
    part["Type"] = "Snapshot"
    part_dif["Type"] = "Dif"    
    
    # Create updated order book
    if len(part) != 0:  
        part_dif = pd.concat([part[["Asks","Bids", "Host", "Pair", "Type"]], part_dif[["Asks","Bids", "Host", "Pair", "Type"]]]).sort_index()
    else:
        part_dif = part_dif[["Asks","Bids", "Host", "Pair", "Type"]]
    part_dif["U_Asks"] = ""
    part_dif["U_Bids"] = ""
    
    if type(last_row) != type(None):
        part_dif = pd.concat([pd.DataFrame(last_row).T, part_dif])  
        part_dif.iat[0,4] = "Previous"   
        
    for i in range(len(part_dif)):
        if part_dif.iloc[i,4] == "Snapshot":
            part_dif.iat[i,5] = part_dif.iloc[i,0].copy()
            part_dif.iat[i,6] = part_dif.iloc[i,1].copy()
        elif part_dif.iloc[i,4] == "Dif":            
            part_dif.iat[i,5] = part_dif.iloc[i-1,5].copy()
            part_dif.iat[i,6] = part_dif.iloc[i-1,6].copy()
            for element in part_dif.iat[i,0]:
                if element != "[]":
                    if element[0] in part_dif.iloc[i,5] and element[1] == '0.000000':
                        del part_dif.iloc[i,5][element[0]]
                    else:
                        part_dif.iat[i,5][element[0]] = element[1]
            for element in part_dif.iat[i,1]:
                if element != "[]":
                    if element[0] in part_dif.iloc[i,6] and element[1] == '0.000000':
                        del part_dif.iloc[i,6][element[0]]
                    else:
                        part_dif.iat[i,6][element[0]] = element[1]
    lrow = part_dif.iloc[-1,:] 
    return part_dif.iloc[1::,:], lrow

# 2.4 Bitstamp

In [30]:
# Timestamp in message
def create_Bitstamp(part, part_dif):
    # First updates

    # Evaluate strings
    part.loc[:,"Asks"] =  part.loc[:,"Asks"].apply(lambda x: dict([[i[0],i[1]] for i in ast.literal_eval(x)]) if x != "-" else x)
    part.loc[:,"Bids"] =  part.loc[:,"Bids"].apply(lambda x: dict([[i[0],i[1]] for i in ast.literal_eval(x)]) if x != "-" else x)
    
    # Create updated order book

    part_join = part.rename(columns = {"Asks":"U_Asks", "Bids":"U_Bids"})
    return part_join

# 2.5 Coinbase

In [31]:
# Timestamp in message
def create_Coinbase(part, part_dif, last_row):

    # Evaluate strings
    part_dif.loc[:,"Asks"] = part_dif.loc[:,"Asks"].apply(lambda x: ast.literal_eval(x) if x != "-" else x)
    part_dif.loc[:,"Bids"] = part_dif.loc[:,"Bids"].apply(lambda x: ast.literal_eval(x) if x != "-" else x)
    part.loc[:,"Asks"] =  part.loc[:,"Asks"].apply(lambda x:dict(ast.literal_eval(x)) if x != "-" else x)
    part.loc[:,"Bids"] =  part.loc[:,"Bids"].apply(lambda x: dict(ast.literal_eval(x)) if x != "-" else x)
    
    part_dif = pd.concat([part[["Asks","Bids", "Host", "Pair"]], part_dif[["Asks","Bids", "Host", "Pair"]]]).sort_index()
    
    part_dif = part_dif.loc[part_dif[part_dif["Asks"] != "-"].index[0]::,:]
    #   In case we want to change indexes
    # part_dif.index.values[i]
    # Rebuild order book for each timestemp we have

    part_dif["U_Asks"] = ""
    part_dif["U_Bids"] = ""
    part_dif.iat[0,4] = part.iloc[0,0].copy()
    part_dif.iat[0,5] = part.iloc[0,1].copy()
    if type(last_row) != type(None):
        part_dif = pd.concat([pd.DataFrame(last_row).T, part_dif])      
    current = part_dif.iloc[0,3]
    for i in range(1,len(part_dif)):
        part_dif.iat[i,4] = part_dif.iloc[i-1,4].copy()
        part_dif.iat[i,5] = part_dif.iloc[i-1,5].copy()
        if part_dif.iloc[i,0] == "-" and part_dif.iloc[i-1,3] == current:
            for element in part_dif.iloc[i,1]:
                if  "sell" == element[0]:
                    if element[1] in part_dif.iloc[i,4]:
                        if element[2] == '0.00000000':
                            del part_dif.iloc[i,4][element[1]]
                        else:
                            part_dif.iat[i,4][element[1]] = element[2]
                    elif element[2] != '0.00000000':
                        part_dif.iat[i,4][element[1]] = element[2]
                else:
                    if element[1] in part_dif.iloc[i,5]:
                        if element[2] == '0.00000000':
                            del part_dif.iloc[i,5][element[1]]
                        else:
                            part_dif.iat[i,5][element[1]] = element[2]
                    elif element[2] != '0.00000000':
                        part_dif.iat[i,5][element[1]] = element[2]          
        elif part_dif.iloc[i,0] != "-":
            part_dif.iat[i,4] = part_dif.iloc[i,0].copy()
            part_dif.iat[i,5] = part_dif.iloc[i,1].copy()
            current = part_dif.iloc[0,3]
        else:
            part_dif.iat[i,4] = part_dif.iloc[i-1,4].copy()
            part_dif.iat[i,5] = part_dif.iloc[i-1,5].copy()
    
    lrow = part_dif.iloc[-1,:]
    return part_dif.iloc[1::,:], lrow

# 2.6 Huobi

In [32]:
# Timestamp in message
def create_Huobi(part, part_dif):
    # First updates
    part.loc[:,"Asks"] = part.loc[:,"Asks"].apply(lambda x: dict(ast.literal_eval(x)) if x != "-" else x)
    part.loc[:,"Bids"] = part.loc[:,"Bids"].apply(lambda x: dict(ast.literal_eval(x)) if x != "-" else x)  
    part = part.rename(columns = {"Asks": "U_Asks", "Bids": "U_Bids"})
    return part

# 2.7 Kraken

In [33]:
# Timestamp in message
# Careful with "r" for republished updates!
def create_Kraken(part, part_dif, last_row):
    part.at[:,"Host"] = "Snapshot"
    
    part_dif = pd.concat([part[["Asks","Bids", "Host", "Pair"]], part_dif[["Asks","Bids", "Host", "Pair"]]]).sort_index()
    part.loc[:,"Asks"] = part.loc[:,"Asks"].apply(lambda x: dict([[i[0],i[1]] for i in ast.literal_eval(x)]) if x != "-" else x)
    part.loc[:,"Bids"] = part.loc[:,"Bids"].apply(lambda x: dict([[i[0],i[1]] for i in ast.literal_eval(x)]) if x != "-" else x)  
    part_dif.loc[:,"Asks"] =  part_dif.loc[:,"Asks"].apply(lambda x: dict([[i[0],i[1]] for i in ast.literal_eval(x)]) if x != "-" else x)
    part_dif.loc[:,"Bids"] =  part_dif.loc[:,"Bids"].apply(lambda x: dict([[i[0],i[1]] for i in ast.literal_eval(x)]) if x != "-" else x)        # Rebuild order book for each timestemp we have

    part_dif = part_dif.loc[part_dif[part_dif["Host"] == "Snapshot"].index[0]::,:]

    part_dif["U_Asks"] = ""
    part_dif["U_Bids"] = ""
    part_dif.iat[0,4] = part.iloc[0,0].copy()
    part_dif.iat[0,5] = part.iloc[0,1].copy()
    if type(last_row) != type(None):
        part_dif = pd.concat([pd.DataFrame(last_row).T, part_dif])   

    current = part_dif.iloc[0,3]
    for i in range(1,len(part_dif)):
        if part_dif.iloc[i,2] != "Snapshot" and part_dif.iloc[i-1,3] == current:
            for j in range(2):
                part_dif.iat[i,4+j] = part_dif.iloc[i-1,4+j].copy()
                if part_dif.iloc[i,j] != "-":
                    for key in part_dif.iloc[i,j].keys():
                        if  key in part_dif.iloc[i,4+j]:
                            if float(part_dif.iloc[i,j][key][0]) == 0.0:
                                del part_dif.iloc[i,4+j][key]
                            else:
                                part_dif.iat[i,4+j][key] = part_dif.iloc[i,j][key][0]
                        else:
                            part_dif.iat[i,4+j][key] = part_dif.iloc[i,j][key][0]        
        elif part_dif.iloc[i,2] == "Snapshot":
            part_dif.iat[i,4] = part_dif.iloc[i,0].copy()
            part_dif.iat[i,5] = part_dif.iloc[i,1].copy()
            current = part_dif.iloc[0,3]
        else:
            part_dif.iat[i,4] = part_dif.iloc[i-1,4].copy()
            part_dif.iat[i,5] = part_dif.iloc[i-1,5].copy()
       
    lrow = part_dif.iloc[-1,:]
    
    return part_dif.iloc[1::,:], lrow

# 3.- General Functions

In [34]:
def compute_stats(ex, part_dif):
    # Compute orderbook price to add to the main dataset
    first = 0
    second = 0
    part_dif["OrderBookPrice " + ex] = ""
    indb = part_dif.columns.get_loc("U_Bids")
    inda = part_dif.columns.get_loc("U_Asks")
    indf = part_dif.columns.get_loc("OrderBookPrice " + ex) 
    for i in range(len(part_dif)):

        values = sorted(part_dif.iloc[i,inda].keys())
        values2 = sorted(part_dif.iloc[i,indb].keys(), reverse = True)
        for j in values:
            if float(part_dif.iloc[i,inda][j]) > 0:
                first = float(j)
                break
        for j in values2:
            if float(part_dif.iloc[i,indb][j]) > 0:
                second = float(j)
                break
        part_dif.iat[i,indf] = (first + second) / 2

    return part_dif

In [35]:
def join_main(ex, df2, part_join):
    j  = part_join[["OrderBookPrice " + ex, "U_Asks", "U_Bids"]].rename(columns = {"U_Asks": "U_Asks " + ex, "U_Bids": "U_Bids " + ex})
    df2 = pd.merge(df2,j, how='outer', left_index=True, right_index=True)
    filter_col = [col for col in df2 if (col.startswith('Last') or col.startswith('Order')
                                         or col.startswith("U"))]
    df2.update(df2[filter_col].ffill().bfill())
    filter_col = [col for col in df2 if (col.startswith('Price') or col.startswith('Q'))]
    df2.update(df2[filter_col].fillna(0))
    
    return df2

In [36]:
def create_OrderBook(ex, df2, part,part_dif, last_row = None):
    if ex == "Bitstamp":
        part_dif = create_Bitstamp(part, part_dif)
    elif ex == "Bitfinex":
        part_dif, last_row = create_Bitfinex(part, part_dif, last_row)
    elif ex == "Binance":
        part_dif, last_row = create_Binance(part, part_dif, last_row)
    elif ex == "Huobi":
        part_dif = create_Huobi(part, part_dif)
    elif ex == "Kraken":
        part_dif, last_row = create_Kraken(part, part_dif, last_row)
    elif ex == "Coinbase":
        part_dif, last_row = create_Coinbase(part, part_dif, last_row)
    elif ex == "Bithumb":
        part_dif, last_row = create_Bithumb(part, part_dif, last_row)
    
    part_dif = compute_stats(ex, part_dif)
    df2 = join_main(ex, df2,part_dif)
    
    return df2, last_row

In [37]:
def create_core(df, pair, exchanges, lrow):
    df1 = df[df["Pair"] == pair]
    
    df2 = df1[df1["Host"] == exchanges[0]].groupby(pd.Grouper(freq = "ms")).agg({"Price":[np.mean],"Q":[np.sum]}).fillna(method = "ffill")
    df2.columns = df2.columns.droplevel(level = 0)
    df2 = df2.rename(columns = {"mean": "LastTrade " + exchanges[0], "sum": "Q " + exchanges[0]})
    for i in exchanges[1::]:
        dft = df1[df1["Host"] == i].groupby(pd.Grouper(freq = "ms")).agg({"Price":[np.mean],"Q":[np.sum]}).fillna(method = "ffill")
        df2[["LastTrade " + i, "Q "+i]] = dft[["Price", "Q"]]
        
    if type(lrow) != type(None):
        df2 = pd.concat([pd.DataFrame(lrow).T, df2])
        
    col1 = [col for col in df2 if (col.startswith('Last'))]
    df2.update(df2[col1].ffill().bfill())
    col2 = [col for col in df2 if (col.startswith('Q'))]
    df2.update(df2[col2].fillna(0))
    
    df2.drop_duplicates(keep = "first", inplace = True)
    
    return df2

In [38]:
def query(date_init, date_end):
    
    points = client.query("SELECT * FROM Price WHERE time >= '"+ str(date_init) +"' and time < '" + str(date_end) + "'")
    points_book = client.query("SELECT * FROM Book WHERE time >= '"+ str(date_init) +"' and time < '" + str(date_end) + "'")
    points_dif = client.query("SELECT * FROM difBook WHERE time >= '"+ str(date_init) +"' and time < '" + str(date_end) + "'")

    last_trades = points["Price"]
    book_snap = points_book["Book"]
    book_dif = points_dif["difBook"]
    
    return last_trades,book_snap,book_dif

In [39]:
def last_metrics_rmdups(df):
    # Compute the max difference and other metrics and remove duplicates
    filter_col = [col for col in df if col.startswith('Order')]
    df["Maximum"] = df[filter_col].apply(lambda row: np.max([np.max([np.abs(row[filter_col[j]]-row[i]) for i in filter_col[j+1::]]) for j in range(len(filter_col)-1)]), axis=1)
    return df

# 4 Final dataframe process

## 4.1 Connect

In [40]:
client = DataFrameClient('localhost', 8086, 'root', 'root')
client.switch_database("SecondM")

## 4.2 Main core

In [52]:
hours_number = 1
interval = 5

start_time = datetime.strptime("2020-05-23 17:30:00.000000", "%Y-%m-%d %H:%M:%S.%f")
endtime = start_time + timedelta(minutes = interval)

end_time = start_time + timedelta(minutes = 5)

pairs = ["ethbtc"]
# pairs = ["ethbtc", "btcusd", "ethusd", "xtzusd"]

exchanges_snap = ["Binance", "Bitfinex", "Bithumb", "Bitstamp", "Coinbase", "Huobi", "Kraken"]

analysis_start = None

max_dif = 0
time_max_dif = 0

last_rows = [None]*8

In [53]:
last_trades, book_snap, book_dif = query(start_time, endtime)

In [64]:
type(last_trades.iloc[0,0]) == type(None)

True

In [68]:
last_trades[~last_trades["Asks"].isnull()]

,Asks,Batch_ID,Bids,Host,Pair,Price,Q,Seg
2020-05-23 17:30:14.862000128+00:00,170.27423328,1154,0.02254,Bitfinex,ethbtc2,NaN,NaN,211
2020-05-23 17:30:14.862000128+00:00,39.199569,1153,0.02255,Bitfinex,ethbtc2,NaN,NaN,109
2020-05-23 17:30:15.084000+00:00,27.30160039,1157,0.0223,Bitfinex,ethbtc2,NaN,NaN,5
2020-05-23 17:30:15.084000+00:00,39.18400855,1155,0.02255,Bitfinex,ethbtc2,NaN,NaN,109
2020-05-23 17:30:15.084000+00:00,7.55032086,1156,0.02231,Bitfinex,ethbtc2,NaN,NaN,4
...,...,...,...,...,...,...,...,...
2020-05-23 17:34:58.375000064+00:00,-54.62640568,1689,0.02257,Bitfinex,ethbtc2,NaN,NaN,10
2020-05-23 17:34:58.375000064+00:00,-61.10114433,1690,0.02258,Bitfinex,ethbtc2,NaN,NaN,70
2020-05-23 17:34:58.596999936+00:00,180.69633374,1692,0.02249,Bitfinex,ethbtc2,NaN,NaN,9
2020-05-23 17:34:58.596999936+00:00,269.05450713,1691,0.0225,Bitfinex,ethbtc2,NaN,NaN,4


In [ ]:
# Check disconnections!

# TO DO, check disconnections for start and end times. Add field in new table for sequence.
# REMMEMBER TO DELETE THE FIRST ROW BEFORE REMOVING DUPLICATES

# For every hour
start = time.time()

while start_time <= end_time:

    # We query the info we need
    if type(last_rows[7]) != type(None):
        lr = last_rows[7][0:14]
    else:
        lr = last_rows[7]
        
    last_trades, book_snap, book_dif = query(start_time, endtime)

    # We create the base dataset
    core_df = create_core(last_trades, pairs[0],exchanges_snap,lr)

    # We add orderbook data
    for k in range(len(exchanges_snap)):
#         print(exchanges_snap[k])
        part = book_snap[book_snap["Host"] == exchanges_snap[k]]
        part = part[part["Pair"].str.contains(pairs[0])].loc[:, part.columns != 'Batch_ID']
        part = part[part["Pair"] == pairs[0]]
        
        part_dif = book_dif[book_dif["Host"] == exchanges_snap[k]]
        part_dif = part_dif[part_dif["Pair"].str.contains(pairs[0])].loc[:, part_dif.columns != 'Batch_ID']
        part_dif = part_dif[part_dif["Pair"] == pairs[0]]

        [core_df, last_rows[k]] = create_OrderBook(exchanges_snap[k], core_df, part, part_dif,last_rows[k])

    # Compute last metrics
    core_df = last_metrics_rmdups(core_df)
    
    if np.max(core_df["Maximum"]) > max_dif:
        max_dif = np.max(core_df["Maximum"]) 
        time_max_dif = start_time
        
    # Write to database
    print(" Writing to dataframe with start time: " + str(start_time))
#     client.write_points(core_df.iloc[1::,:],"Dataset",time_precision = "n", database = "SecondM", batch_size = 500)
    
    # We set up the interval times
    start_time = endtime
    endtime = start_time + timedelta(minutes = interval)
    if core_df.isna().sum().sum() == 0 and type(analysis_start) != type(None):
        analysis_start = endtime
        
    last_rows[7] = core_df.iloc[-1,:]
    
end = start - time.time()

print("Total time: " + str(end))

# DRAFT

In [16]:
client = DataFrameClient('localhost', 8086, 'root', 'root')
client.switch_database("SecondM")

In [26]:
start_time = "2020-05-20 23:35:00.000000"
end_time = datetime.strptime(start_time, "%Y-%m-%d %H:%M:%S.%f") + timedelta(minutes = 30)
hours_number = 2
intervals = hours_number
# pairs = ["ethbtc", "btcusd", "ethusd", "xtzusd"]
pairs = ["ethbtc"]
results = []

In [27]:
j = 0
i = 0

In [30]:
# We query the info we need
last_trades, book_snap, book_dif, exchanges_trades, exchanges_snap = query(start_time, end_time)


In [31]:
exchanges_trades

array(['Binance', 'Bitfinex', 'Bithumb', 'Bitstamp', 'Coinbase', 'Huobi',
       'Kraken'], dtype=object)

In [ ]:
# We create the base dataset
core_df = create_core(last_trades, "ethbtc")

In [470]:
k = exchanges_trades[6]
p = "ethbtc"

In [515]:
part = book_snap[book_snap["Host"] == k]
part = part[part["Pair"].str.contains(p)].loc[:, part.columns != 'Batch_ID']
part_dif = book_dif[book_dif["Host"] == k]
part_dif = part_dif[part_dif["Pair"].str.contains(p)].loc[:, part_dif.columns != 'Batch_ID']

In [247]:
data = client.query("SELECT * FROM difBook WHERE time > '2020-05-20 23:35:00.000000' and time < '2020-05-21 00:05:00.000000'")

In [ ]:
# Binance_ETHBTC : 371; 6; 157
# Bitfinex_ETHBTC : 97; 7; 249
# Bitstamp_ETHBTC : 4; 84; 0
# Bithumb_ETHBTC : 110; 1; 151
# Coinbase_ETHBTC : 16; 1; 375
# Huobi_ETHBTC : 83; 154; 0
# Kraken_ETHBTC : 146; 1; 416

In [701]:
np.unique(data["difBook"]["Pair"])

array(['ethbtc', 'ethbtc2', 'ethbtc3'], dtype=object)

In [702]:
np.unique(data["difBook"]["Host"])

array(['Binance', 'Bitfinex', 'Bithumb', 'Coinbase', 'Kraken'],
      dtype=object)

In [703]:
np.unique(data["difBook"][data["difBook"]["Host"] == "Coinbase"]["Pair"], return_counts = True)

(array(['ethbtc', 'ethbtc2', 'ethbtc3'], dtype=object),
 array([1090,  963,  748], dtype=int64))

In [54]:
data["difBook"][data["difBook"]["Host"] == "Bitfinex"]

NameError: name 'data' is not defined

In [248]:
data["difBook"][(data["difBook"]["Host"] == "Bitfinex") & (data["difBook"]["Pair"] == "ethbtc")].groupby(pd.Grouper(freq = "min")).count()

,Asks,Batch_ID,Bids,Host,Pair,Seg
2020-05-20 23:39:00+00:00,66,66,66,66,66,66
2020-05-20 23:40:00+00:00,290,290,290,290,290,290
2020-05-20 23:41:00+00:00,186,186,186,186,186,186
2020-05-20 23:42:00+00:00,151,151,151,151,151,151
2020-05-20 23:43:00+00:00,189,189,189,189,189,189
2020-05-20 23:44:00+00:00,99,99,99,99,99,99
2020-05-20 23:45:00+00:00,188,188,188,188,188,188
2020-05-20 23:46:00+00:00,193,193,193,193,193,193
2020-05-20 23:47:00+00:00,0,0,0,0,0,0
2020-05-20 23:48:00+00:00,220,220,220,220,220,220


In [249]:
data["difBook"][(data["difBook"]["Host"] == "Bitfinex") & (data["difBook"]["Pair"] == "ethbtc2")].groupby(pd.Grouper(freq = "min")).count()

,Asks,Batch_ID,Bids,Host,Pair,Seg
2020-05-20 23:40:00+00:00,172,172,172,172,172,172
2020-05-20 23:41:00+00:00,161,161,161,161,161,161
2020-05-20 23:42:00+00:00,289,289,289,289,289,289
2020-05-20 23:43:00+00:00,104,104,104,104,104,104
2020-05-20 23:44:00+00:00,124,124,124,124,124,124
2020-05-20 23:45:00+00:00,139,139,139,139,139,139
2020-05-20 23:46:00+00:00,89,89,89,89,89,89
2020-05-20 23:47:00+00:00,201,201,201,201,201,201
2020-05-20 23:48:00+00:00,48,48,48,48,48,48
2020-05-20 23:49:00+00:00,216,216,216,216,216,216


In [250]:
data["difBook"][(data["difBook"]["Host"] == "Bitfinex") & (data["difBook"]["Pair"] == "ethbtc3")].groupby(pd.Grouper(freq = "min")).count()

,Asks,Batch_ID,Bids,Host,Pair,Seg
2020-05-20 23:40:00+00:00,11,11,11,11,11,11
2020-05-20 23:41:00+00:00,229,229,229,229,229,229
2020-05-20 23:42:00+00:00,116,116,116,116,116,116
2020-05-20 23:43:00+00:00,191,191,191,191,191,191
2020-05-20 23:44:00+00:00,240,240,240,240,240,240
2020-05-20 23:45:00+00:00,0,0,0,0,0,0
2020-05-20 23:46:00+00:00,212,212,212,212,212,212
2020-05-20 23:47:00+00:00,81,81,81,81,81,81
2020-05-20 23:48:00+00:00,132,132,132,132,132,132
2020-05-20 23:49:00+00:00,134,134,134,134,134,134


In [698]:
data2 = client.query("SELECT * FROM Book WHERE time > '2020-05-20 20:55:00.000000' and time < '2020-05-20 21:50:00.000000'")

In [699]:
data2["Book"][data2["Book"]["Host"] == "Coinbase"]

,Asks,Batch_ID,Bids,Host,LastUpdateID,Pair
2020-05-20 21:01:24.599000064+00:00,"[['0.02202', '65.38005001'], ['0.02203', '52.9...",0,"[['0.02201', '60.75720772'], ['0.02200', '130....",Coinbase,-,ethbtc
2020-05-20 21:01:48.633999872+00:00,"[['209.90', '20.37292661'], ['209.91', '1.4097...",0,"[['209.89', '13.04532467'], ['209.82', '50.517...",Coinbase,-,ethusd
2020-05-20 21:02:13.836000+00:00,"[['9529.93', '0.13285832'], ['9531.28', '0.198...",0,"[['9529.92', '0.10000000'], ['9529.59', '0.161...",Coinbase,-,btcusd
2020-05-20 21:02:30.195000064+00:00,"[['0.02202', '64.43155475'], ['0.02203', '66.1...",0,"[['0.02201', '44.97981892'], ['0.02200', '125....",Coinbase,-,ethbtc2
2020-05-20 21:03:11.047000064+00:00,"[['0.02201', '45.29779780'], ['0.02202', '69.4...",0,"[['0.02200', '71.33930905'], ['0.02199', '76.0...",Coinbase,-,ethbtc3
2020-05-20 21:03:37.969999872+00:00,"[['209.77', '7.73127497'], ['209.79', '2.47694...",0,"[['209.76', '69.68934529'], ['209.73', '154.30...",Coinbase,-,ethusd2
2020-05-20 21:03:49.339000064+00:00,"[['0.02201', '89.47313018'], ['0.02202', '67.6...",1,"[['0.02200', '23.04976749'], ['0.02199', '41.1...",Coinbase,-,ethbtc
2020-05-20 21:04:14.476000+00:00,"[['9526.15', '5.21003725'], ['9526.77', '0.040...",0,"[['9526.14', '0.09190600'], ['9526.06', '0.066...",Coinbase,-,btcusd2
2020-05-20 21:04:15.104999936+00:00,"[['209.57', '7.26484141'], ['209.59', '8.95085...",0,"[['209.56', '65.08430046'], ['209.55', '1.2650...",Coinbase,-,ethusd3
2020-05-20 21:04:37.415000064+00:00,"[['0.02201', '92.37238554'], ['0.02202', '136....",1,"[['0.02200', '0.64976749'], ['0.02199', '44.04...",Coinbase,-,ethbtc2


In [406]:
# exchange = ccxt.coinbasepro()

In [417]:
# for i in range(3):
#     result = await exchange.fetch_order_book("ETH/BTC")
#     print(result)
#     time.sleep(2)